In [70]:
import yfinance as yf
import pandas as pd

### Coleta e preparação dos dados

Nesta etapa são coletados os preços ajustados dos ativos via Yahoo Finance.<br>
A análise é conduzida em frequência mensal, pois o objetivo do projeto é o
monitoramento de regimes de longo prazo, e não a reação a ruídos de curto prazo.

Cada ativo será analisado individualmente, sempre em relação ao seu próprio
histórico, evitando comparações cruzadas entre ativos diferentes.
<br>
Escolhemos 3 ativos aleatóriamente para compor a carteira a ser analisada: PETRA4, VALE3 e ITSA4.

In [71]:

tickers = [
    "PETR4.SA",
    "VALE3.SA",
    "ITSA4.SA"
]

precos_mensais = {}

for ticker in tickers:
    dados = yf.download(
        ticker,
        start="2000-01-01",
        progress=False,
        auto_adjust=True
    )

    if dados.empty:
        print(f"Aviso: sem dados para {ticker}")
        continue

    # EXTRAI A SERIES CORRETA DO MULTIINDEX
    close = dados[("Close", ticker)]

    # REMOVE LINHAS BUGADAS (preço zero)
    close = close[close > 0]

    # GARANTE ÍNDICE DATETIME
    close.index = pd.to_datetime(close.index)

    # CONVERTE PARA MENSAL
    close_mensal = close.resample("ME").last()

    # VALIDA
    if close_mensal.size > 1:
        precos_mensais[ticker] = close_mensal
    else:
        print(f"Aviso: série mensal inválida para {ticker}")

# CONCATENA CORRETAMENTE
df_precos = pd.concat(precos_mensais, axis=1)

print(df_precos.tail())


             PETR4.SA   VALE3.SA   ITSA4.SA
Date                                       
2025-09-30  30.505873  54.715580  10.503035
2025-10-31  28.847736  62.013527  10.667858
2025-11-30  30.825867  64.047066  11.290532
2025-12-31  30.820000  71.959999  11.680000
2026-01-31  37.070000  86.570000  13.920000


Após o download dos dados e sua agrupação por mês, pois nos interessa olhar o fechamento do mês, nós convertemos os valores brusto para percentuais com a função pct_change().

In [72]:
df_retornos = df_precos.pct_change()

print(df_retornos.tail())


            PETR4.SA  VALE3.SA  ITSA4.SA
Date                                    
2025-09-30  0.011575  0.036357  0.024430
2025-10-31 -0.054355  0.133380  0.015693
2025-11-30  0.068571  0.032792  0.058369
2025-12-31 -0.000190  0.123549  0.034495
2026-01-31  0.202790  0.203029  0.191781


### Z-score absoluto

O z-score absoluto mede o quão anormal foi o retorno mensal em relação ao próprio
histórico recente do ativo. Para isso, utiliza-se uma janela móvel de 24 meses,
que representa aproximadamente dois anos de comportamento recente.

Essa métrica não mede retorno, mas sim desvio em relação ao padrão histórico,
permitindo identificar meses estatisticamente fracos ou fortes.


In [73]:
# Média móvel de 24 meses
media_24m = df_retornos.rolling(window=24, min_periods=24).mean()

# Desvio padrão móvel de 24 meses (amostral)
std_24m = df_retornos.rolling(window=24, min_periods=24).std()

# Z-score mensal
zscore = (df_retornos - media_24m) / std_24m



Agora iniciamos o download dos dados da ibovespa e o mesmo processo se repete, até a criação do Zscore relativo.

In [74]:
# Download do Ibovespa
ibov = yf.download(
    "^BVSP",
    start="2000-01-01",
    progress=False,
    auto_adjust=True
)

# Série de fechamento ajustado
ibov_close = ibov["Close"]
ibov_close.index = pd.to_datetime(ibov_close.index)

# 🔴 USAR O MESMO RESAMPLE QUE OS ATIVOS
ibov_mensal = ibov_close.resample("ME").last()

# Retorno mensal do Ibovespa
ibov_retornos = ibov_mensal.pct_change()


In [75]:
ibov_retornos = ibov_retornos.squeeze()

In [76]:
# Alinhar índices explicitamente
ibov_retornos = ibov_retornos.loc[df_retornos.index]


### Z-score relativo ao Ibovespa

Além da análise absoluta, é calculado um z-score relativo ao Ibovespa.
Essa métrica permite distinguir se um desempenho fraco do ativo é resultado de
uma deterioração própria ou apenas reflexo de um movimento geral do mercado.
<br><br>
O foco não é medir correlação, mas identificar desvios anormais do
comportamento relativo do ativo ao longo do tempo.


In [77]:
# Retorno relativo
retorno_relativo = df_retornos.sub(ibov_retornos, axis=0)

# Rolling 24 meses
media_rel_24m = retorno_relativo.rolling(24, min_periods=24).mean()
std_rel_24m   = retorno_relativo.rolling(24, min_periods=24).std()

# Z-score relativo
zscore_rel = (retorno_relativo - media_rel_24m) / std_rel_24m


### Definição do regime de baixo desempenho

O regime de baixo desempenho é definido de forma estatística e antecede o uso de
Machine Learning. Um ativo entra em regime fraco apenas quando apresenta dois
meses consecutivos de retorno abaixo da sua média histórica recente.
<br><br>

Essa definição evita reagir a quedas pontuais e garante que o modelo aprenda
padrões associados à persistência da deterioração, e não a ruído.



In [79]:
# Sinal fraco mensal: 1 se z-score < 0, senão 0
sinal_fraco = (zscore < 0).astype(int)


In [80]:
sinal_fraco.tail()


,PETR4.SA,VALE3.SA,ITSA4.SA
Date,,,
2025-09-30,0,0,0
2025-10-31,1,0,1
2025-11-30,0,0,0
2025-12-31,1,0,0
2026-01-31,0,0,0


In [81]:
# Label final: persistência de 2 meses
label = ((sinal_fraco == 1) & (sinal_fraco.shift(1) == 1)).astype(int)


In [82]:
label.tail()


,PETR4.SA,VALE3.SA,ITSA4.SA
Date,,,
2025-09-30,0,0,0
2025-10-31,0,0,0
2025-11-30,0,0,0
2025-12-31,0,0,0
2026-01-31,0,0,0


### Construção das variáveis explicativas

As variáveis explicativas foram escolhidas para capturar diferentes dimensões do
comportamento recente do ativo: nível, tendência, volatilidade e anomalia.<br><br>

São utilizados múltiplos horizontes temporais e defasagens para evitar qualquer
uso de informação futura, preservando a coerência temporal do modelo.<br><br>


In [83]:
# Converte retornos para formato longo
df_long = (
    df_retornos
    .stack()
    .reset_index()
    .rename(columns={
        "Date": "data",
        "level_1": "ativo",
        0: "retorno"
    })
)


In [85]:
# Ordenar corretamente
df_long = df_long.sort_values(["ativo", "data"])

# Retorno acumulado 3 e 6 meses
df_long["ret_3m"] = (
    df_long
    .groupby("ativo")["retorno"]
    .rolling(3)
    .apply(lambda x: (1 + x).prod() - 1, raw=False)
    .reset_index(level=0, drop=True)
)

df_long["ret_6m"] = (
    df_long
    .groupby("ativo")["retorno"]
    .rolling(6)
    .apply(lambda x: (1 + x).prod() - 1, raw=False)
    .reset_index(level=0, drop=True)
)

# Volatilidade 6 meses
df_long["vol_6m"] = (
    df_long
    .groupby("ativo")["retorno"]
    .rolling(6)
    .std()
    .reset_index(level=0, drop=True)
)


Z-scores defasados (evitar vazamento)

In [86]:
# Z-score absoluto defasado
zscore_long = zscore.stack().reset_index(name="zscore")
zscore_long["zscore_lag1"] = zscore_long.groupby("level_1")["zscore"].shift(1)

# Z-score relativo defasado
zscore_rel_long = zscore_rel.stack().reset_index(name="zscore_rel")
zscore_rel_long["zscore_rel_lag1"] = zscore_rel_long.groupby("level_1")["zscore_rel"].shift(1)


Merge das features

In [87]:
# Merge z-score absoluto
df_long = df_long.merge(
    zscore_long[["Date", "level_1", "zscore_lag1"]],
    left_on=["data", "ativo"],
    right_on=["Date", "level_1"],
    how="left"
).drop(columns=["Date", "level_1"])

# Merge z-score relativo
df_long = df_long.merge(
    zscore_rel_long[["Date", "level_1", "zscore_rel_lag1"]],
    left_on=["data", "ativo"],
    right_on=["Date", "level_1"],
    how="left"
).drop(columns=["Date", "level_1"])


In [88]:
df_long.head(10)

,data,ativo,retorno,ret_3m,ret_6m,vol_6m,zscore_lag1,zscore_rel_lag1
0,2000-02-29,ITSA4.SA,-0.047058,NaN,NaN,NaN,NaN,NaN
1,2000-03-31,ITSA4.SA,0.049382,NaN,NaN,NaN,NaN,NaN
2,2000-04-30,ITSA4.SA,-0.088234,-0.088234,NaN,NaN,NaN,NaN
3,2000-05-31,ITSA4.SA,-0.038711,-0.080248,NaN,NaN,NaN,NaN
4,2000-06-30,ITSA4.SA,0.161074,0.017647,NaN,NaN,NaN,NaN
5,2000-07-31,ITSA4.SA,0.034683,0.154838,0.052941,0.089712,NaN,NaN
6,2000-08-31,ITSA4.SA,0.078212,0.295302,0.191357,0.087816,NaN,NaN
7,2000-09-30,ITSA4.SA,-0.031087,0.080926,0.100001,0.090857,NaN,NaN
8,2000-10-31,ITSA4.SA,-0.139038,-0.100559,0.038708,0.104263,NaN,NaN
9,2000-11-30,ITSA4.SA,-0.006212,-0.170985,0.073825,0.101993,NaN,NaN


In [89]:
# Remover linhas com NaN (início das janelas)
df_modelo = df_long.dropna().reset_index(drop=True)


Até aqui o modelo está quase pronto faltando apenas a variável alvo.
Adicionando a vairável alvo.


In [90]:
label_long = (
    label
    .stack()
    .reset_index()
    .rename(columns={
        "Date": "data",
        "level_1": "ativo",
        0: "label"
    })
)

df_modelo = df_modelo.merge(
    label_long,
    on=["data", "ativo"],
    how="inner"
)


In [91]:
df_modelo.head()

,data,ativo,retorno,ret_3m,ret_6m,vol_6m,zscore_lag1,zscore_rel_lag1,label
0,2002-02-28,ITSA4.SA,0.126169,0.131456,0.242268,0.079453,-0.315612,0.518958,0
1,2002-03-31,ITSA4.SA,-0.012448,0.096774,0.329609,0.063134,1.217152,-0.052280,0
2,2002-04-30,ITSA4.SA,-0.025210,0.084112,0.221053,0.069623,-0.345781,0.355100,1
3,2002-05-31,ITSA4.SA,-0.099137,-0.132780,-0.018779,0.073588,-0.537464,-0.729150,1
4,2002-06-30,ITSA4.SA,-0.086125,-0.197479,-0.119815,0.080135,-1.338670,-1.907286,1


In [92]:
# df_modelo = df_modelo.rename(columns={"label_x": "label"})


In [93]:
df_modelo.head()

,data,ativo,retorno,ret_3m,ret_6m,vol_6m,zscore_lag1,zscore_rel_lag1,label
0,2002-02-28,ITSA4.SA,0.126169,0.131456,0.242268,0.079453,-0.315612,0.518958,0
1,2002-03-31,ITSA4.SA,-0.012448,0.096774,0.329609,0.063134,1.217152,-0.052280,0
2,2002-04-30,ITSA4.SA,-0.025210,0.084112,0.221053,0.069623,-0.345781,0.355100,1
3,2002-05-31,ITSA4.SA,-0.099137,-0.132780,-0.018779,0.073588,-0.537464,-0.729150,1
4,2002-06-30,ITSA4.SA,-0.086125,-0.197479,-0.119815,0.080135,-1.338670,-1.907286,1


Treinamento do modole

In [94]:
features = [
    "retorno",
    "ret_3m",
    "ret_6m",
    "vol_6m",
    "zscore_lag1",
    "zscore_rel_lag1"
]

df_modelo = df_modelo.sort_values("data")

X = df_modelo[features]
y = df_modelo["label"]


### Separação temporal dos dados

A separação entre dados de treino e teste é feita de forma temporal, respeitando
a ordem cronológica das observações. Não é utilizado embaralhamento dos dados,
pois isso introduziria vazamento de informação.

Essa abordagem reflete o uso real do sistema, que sempre opera com dados passados
para avaliar estados futuros.


In [95]:
split_date = df_modelo["data"].quantile(0.8)

X_train = X[df_modelo["data"] <= split_date]
X_test  = X[df_modelo["data"] > split_date]

y_train = y[df_modelo["data"] <= split_date]
y_test  = y[df_modelo["data"] > split_date]


### Padronizando os dados
A pradronização faz com que o modelo consiga trabalhar de forma mais adequada com os dados.

In [96]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)


### Escolha do modelo de Machine Learning

A Regressão Logística foi escolhida por sua interpretabilidade, estabilidade e
adequação ao uso como escore probabilístico. O objetivo do modelo não é maximizar
performance preditiva, mas fornecer um sinal auxiliar de fragilidade estatística.

Modelos mais complexos poderiam aumentar a variância sem trazer ganhos reais
para um sistema de monitoramento defensivo.

In [97]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    random_state=42
)

model.fit(X_train_scaled, y_train)


LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)

In [98]:
# Probabilidade de regime (escore mensal)
df_modelo.loc[X_test.index, "score_ml"] = model.predict_proba(X_test_scaled)[:, 1]


### Avaliação do modelo

A avaliação do modelo não é conduzida com foco em métricas tradicionais como
acurácia ou F1-score. O evento de interesse é raro e o score é utilizado como
indicador gradual de risco, não como classificador binário rígido.
<br><br>
A coerência econômica, estabilidade do score e sua utilidade prática ao longo
do tempo são os principais critérios de avaliação.


In [99]:
from sklearn.metrics import classification_report, roc_auc_score

y_pred = model.predict(X_test_scaled)

print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, model.predict_proba(X_test_scaled)[:, 1]))


              precision    recall  f1-score   support

           0       0.98      0.85      0.91       130
           1       0.67      0.95      0.79        41

    accuracy                           0.88       171
   macro avg       0.83      0.90      0.85       171
weighted avg       0.91      0.88      0.88       171

ROC-AUC: 0.95046904315197


In [100]:
import pandas as pd

coeficientes = pd.Series(
    model.coef_[0],
    index=features
).sort_values()

coeficientes


retorno           -3.040787
zscore_lag1       -2.699211
vol_6m            -0.657354
zscore_rel_lag1   -0.234258
ret_3m            -0.104673
ret_6m             0.540140
dtype: float64

### Sobre a célula comentada abaixo:
Ao rodar o notebook você deve habilitar a célula, após isso, volte a comentá-la, para não correr o risco de sobrescrever o modelo.
<br><br>
IMPORTANTE:
O modelo só deve ser salvo novamente caso haja alteração
na definição do label, features ou estrutura do modelo.


In [101]:
# Função para salvar o modelo.

# import joblib
# import os

# # Garantir pasta
# os.makedirs("modelos", exist_ok=True)

# # Salvar scaler e modelo
# joblib.dump(scaler, "modelos/scaler.pkl")
# joblib.dump(model, "modelos/modelo_logistico.pkl")

# print("Modelo e scaler salvos com sucesso.")


### Conclusão e uso prático

Este notebook documenta o processo de construção do sistema de monitoramento de
deterioração de ativos, desde a definição do problema até a geração de um escore
auxiliar via Machine Learning.

O resultado final do trabalho é operacionalizado em um script Python que permite
uso mensal recorrente, com geração de relatórios em Excel e histórico acumulado.
O sistema deve ser utilizado como apoio à decisão, respeitando suas limitações
e o contexto de investimento de longo prazo.
